In [1]:
using Flux
using DifferentialEquations
using Plots

Define the neural network structure

*Need to iterate through different structures

In [2]:
N = 4 #x,theta,dx,dtheta
m = Chain(LSTM(N,256),LSTM(256,256),LSTM(256,N))

Chain(Recur(LSTMCell(4, 256)), Recur(LSTMCell(256, 256)), Recur(LSTMCell(256, 4)))

Develop the data for testing the system

Is this how it should be setup?

How do I train on-line?

In [3]:
function cart(du,u,p,t)

    m = 2
    g = 9.81
    l = 0.5
    M = 5
    J_G = 1/12*m*l^2
    f(u,t) = 0
    torque(u,t) = 0

    du[1] = (J_G*u[3] + l^2*m*u[3] - l*m*u[4]*cos(u[2]))/(J_G*M + J_G*m + M*l^2*m + l^2*m^2*sin(u[2])^2)
    du[2] = M*u[4] - l*m*u[3]*cos(u[2]) + m*u[4]/(J_G*M + J_G*m + M*l^2*m + l^2*m^2*sin(u[2])^2)
    du[3] = f(u,t)
    du[4] = 2*l^2*m^2*(J_G*M*g*l*m*cos(u[2]) + J_G*g*l*m^2*cos(u[2]) + J_G*u[3]^2/2 + M*g*l^3*m^2*cos(u[2]) + M*u[4]^2/2 - g*l^3*m^3*cos(u[2])^3 + g*l^3*m^3*cos(u[2]) + l^2*m*u[3]^2/2 - l*m*u[3]*u[4]*cos(u[2]) + m*u[4]^2/2)*sin(u[2])*cos(u[2])/(J_G*M + J_G*m + M*l^2*m + l^2*m^2*sin(u[2])^2)^2 + torque(u,t) - (-J_G*M*g*l*m*sin(u[2]) - J_G*g*l*m^2*sin(u[2]) - M*g*l^3*m^2*sin(u[2]) + 3*g*l^3*m^3*sin(u[2])*cos(u[2])^2 - g*l^3*m^3*sin(u[2]) + l*m*u[3]*u[4]*sin(u[2]))/(J_G*M + J_G*m + M*l^2*m + l^2*m^2*sin(u[2])^2)

end

t0 = 0.0
tf = 10.0
tspan = (t0,tf)

u0 = [0.0, 45*π/180, 0.0, 0.0]

prob = ODEProblem(cart,u0,tspan)
sol = solve(prob,DP5())
data = [ (sol[i], sol[i+1]) for i in 1:(length(sol)-1) ]

85-element Array{Tuple{Array{Float64,1},Array{Float64,1}},1}:
 ([0.0, 0.785398, 0.0, 0.0], [-1.54447e-8, 0.785399, 0.0, 0.000998729])              
 ([-1.54447e-8, 0.785399, 0.0, 0.000998729], [-1.86877e-6, 0.785447, 0.0, 0.0109862])
 ([-1.86877e-6, 0.785447, 0.0, 0.0109862], [-0.000189852, 0.79037, 0.0, 0.111063])   
 ([-0.000189852, 0.79037, 0.0, 0.111063], [-0.00361234, 0.886194, 0.0, 0.513195])    
 ([-0.00361234, 0.886194, 0.0, 0.513195], [-0.0133798, 1.31552, 0.0, 1.27747])       
 ([-0.0133798, 1.31552, 0.0, 1.27747], [0.00911104, 2.56782, 0.0, 2.27344])          
 ([0.00911104, 2.56782, 0.0, 2.27344], [0.0846933, 3.93696, 0.0, 2.19129])           
 ([0.0846933, 3.93696, 0.0, 2.19129], [0.0963494, 5.05847, 0.0, 1.138])              
 ([0.0963494, 5.05847, 0.0, 1.138], [0.0849616, 5.47931, 0.0, 0.155632])             
 ([0.0849616, 5.47931, 0.0, 0.155632], [0.0953078, 5.10864, 0.0, -1.06165])          
 ([0.0953078, 5.10864, 0.0, -1.06165], [0.0823371, 3.87847, 0.0, -2.21511])   

Define the loss and optimization functions to use. Using mean-squared and gradient descent.

In [4]:
loss(x,y) = Flux.mse(m(x), y)
ps = Flux.params(m)
opt = Descent(0.1)
;

Train the network using the built in function

It looks like it trains for one single touple and then fails.... WHY??

In [5]:
scatter()
for i = 1:(length(sol) - 1)
    x = sol[i]
    y = sol[i+1]
    l = loss(m(x), y)
    
    println(l)
    #Flux.train!(loss,ps,[data[i]],opt,cb = () -> println("Training"))
end

0.19319297323901657 (tracked)
0.25091391124580575 (tracked)
0.31315604548375575 (tracked)
0.5141269924866396 (tracked)
1.2808547037681275 (tracked)
3.7133187184256204 (tracked)
6.054390207417623 (tracked)
7.759518856735535 (tracked)
8.513144712188769 (tracked)
7.588006699970298 (tracked)
5.404003427691151 (tracked)
2.3997743526426527 (tracked)
0.85738317840988 (tracked)
0.3746783777064454 (tracked)
0.6108823991473933 (tracked)
1.7599103585065623 (tracked)
4.080225979311348 (tracked)
6.135394419854081 (tracked)
7.960183777775004 (tracked)
8.409283633920397 (tracked)
7.180752041466035 (tracked)
4.521887659530339 (tracked)
1.9218930117934572 (tracked)
0.571978752202634 (tracked)
0.3865627637801927 (tracked)
0.9366173601674506 (tracked)
3.3465814254864714 (tracked)
6.14897033243989 (tracked)
7.8684629287356636 (tracked)
8.320243023551525 (tracked)
6.948483855979205 (tracked)
4.515330342916984 (tracked)
1.924476676262423 (tracked)
0.5909579119303217 (tracked)
0.39383507069889584 (tracked)
0

Train the network using a custom algorithm